<a href="https://colab.research.google.com/github/lwallac2/Bank-Marketing/blob/main/Module_7_kNN_NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 6: CLASSIFICATION: k Nearest Neighbor and Naive Bayes**
This week, we will step into machine learning with the k Nearest Neighbor and Naive Bayes algorithms. At the end of this week you will be able to:
* Outline the concepts of Supervised Learning
* Explain what classification is
* Describe how k Nearest Neighbor works
* Describe how Naive Bayes works
* Write code to execute both, k Nearest Neighbor and Naive Bayes

##**Supervised Learning**
This week, we are going to take the whole idea of forecasting a step further. With last week's logistic regression, we learned how to build a model that sorts data into 1 of two factors in a target attribute; in other words: A model with binary class outcome. This week, we are going to work with a categorical class attribute and two different machine learning mechanisms: The empirical classifier k Nearest Neighbor and the statistical classifier Naive Bayes.

Both are part of **SUPERVISED LEARNING**. A supervised machine learning algorithm relies on labeled input data to learn a function that produces an appropriate output when given new unlabeled data. Imagine a computer is a child, we are its supervisor (e.g. parent, guardian, or teacher), and we want the child (computer) to learn what a book looks like. We will show the child several different pictures, some of which are books and the rest could be pictures of anything (cats, coffee cups, computers, etc).
When we see a book, we shout "book!” When it’s not a pig, we shout “no, not book!” After doing this several times with the child, we show them a picture and ask “book?” and they will correctly (most of the time) say “book!” or “no, not book!” depending on what the picture is. That is supervised learning.

Now, please watch the video below. It's a great introduction to Supervised Learning.

In [1]:
from IPython.display import HTML # This is just for me so I can embed videos
from IPython.display import Image # This is just for me so I can embed images
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/kE5QZ8G_78c" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

##**Classification**
Classification is the problem of identifying which of a set of categories an observation belongs to. For example: Classification helps us assign a given email to the "spam" or "non-spam" class, or a diagnosis to a given patient based on observed characteristics of the patient. 

In machine learning, classification has these steps:
1. Determine what the class attribute in the dataset should be. This will be the attribute you'll predict later on
2. Preprocess the data (remove n/a, transform data types as needed, deal with missing data) and ensure that the dependent attribute is CATEGORICAL
3. Split the data into a training set and a test set
4. Build the model based on the training set
5. Test the model on the test set and compare the calculated class values to the actual class values shown in the test set.
6. Determine the quality of the model

Ready? Let's go.


#**0. Preparation and Setup**
For these explanations, we will need a model with a dependent attribute that is categorical. The typical explanation uses the famous [iris flower dataset](https://github.com/shstreuber/Data-Mining/blob/master/data/iris.csv), which even has [its own wikipedia page](https://en.wikipedia.org/wiki/Iris_flower_data_set). However, we will use the insurance dataset because it allows us to tackle actual real-world problems. Since we will be working with two different types of classification, the first one called k Nearest Neighbor, and the second on called Naive Bayes, we will import all the libraries upfront.

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import spatial
import statsmodels.api as sm

from IPython.display import HTML # This is just for me so I can embed videos
from IPython.display import Image # This is just for me so I can embed images

#Reading in the data as insurance dataframe
insurance = pd.read_csv("https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/insurance_with_categories.csv")

#Verifying that we can see the data
insurance.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# **1. k Nearest Neighbor**
The concept of the k-nearest neighbor classifier is part of our everyday life: Imagine you meet a group of young, stylish, and sportive people. They talk about their friend Ben, who isn't with them. How do you imagine Ben? Young, stylish, and sportive? Bingo! Then you learn that Ben lives in a neighborhood where people have an average income above 200000 dollars a year and drive fancy European cars. Both his neighbors make even more than 300,000 dollars per year! Most likely, you imagine Ben to drive an Audi or a Porsche, as well, right?

The principle behind nearest neighbor classification consists in finding a predefined number, i.e. the 'k' - of training samples closest in (Euclidian) distance to the new sample that we want to classify. The label of the new sample will be defined based on these neighbors. 

Would you like a more in-depth explanation? The video below gives you all the detail you will want to know as you proceed.


In [3]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/4HKqjENq9OU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

##**1.1 Nearest Neighbor Algorithm**
So, in a nutshell, the k Nearest Neighbor Algorithm works like this:
1. Load the data
2. Initialize K to your chosen number of neighbors
3. For each example in the data: 
  
  3.1 Calculate the distance between the query example and the current example from the data.
  
  3.2 Add the distance and the index of the example to an ordered collection
4. Sort the ordered collection of distances and indices from smallest to largest (in ascending order) by the distances
5. Pick the first K entries from the sorted collection
6. Get the labels of the selected K entries
7. If classification, return the mode of the K labels

To work with the k Nearest Neighbor algorithm, we use its library from the scikit learn package. We will also learn a new way to build training and test sets (with a process called cross-validation), so we are importing that package, too. Lastly, we will be generating "pretty pictures"--so, matplotlib is going to help us out with that.



In [4]:
# We import all the kNN libraries

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

from sklearn import neighbors, datasets
from sklearn.model_selection import cross_val_score, train_test_split
%matplotlib inline 

##**1.2 Exploratory Data Analysis**

1. Let's investigate the features (= attributes or dimensions)

In [5]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


We have 4 numeric attributes and 3 categorical ones. 

2. Let's rearrange the numeric features into a dataframe and use them to predict what region a person comes from.

In [6]:
insurance2 = pd.DataFrame(insurance, columns = ['age', 'bmi', 'children','charges','region'])
insurance2.head()

,age,bmi,children,charges,region
0,19,27.900,0,16884.92400,southwest
1,18,33.770,1,1725.55230,southeast
2,28,33.000,3,4449.46200,southeast
3,33,22.705,0,21984.47061,northwest
4,32,28.880,0,3866.85520,northwest


In [7]:
# Let's check what the levels of region are!
insurance2.region.unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

And now for some fun stuff! Pandas has a great package that makes EDA much much easier: The pandas profiling package. It's not a standard package, so you'll need to install it with pip, then load it, and finally call it. Click around the report to learn more about the dataset we just built!

In [8]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip # Installing the package first

     / 25.9 MB 9.6 MB/s
     |████████████████████████████████| 10.9 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 102 kB 13.0 MB/s 
     |████████████████████████████████| 675 kB 60.6 MB/s 
     |████████████████████████████████| 4.7 MB 50.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 812 kB 55.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py3-none-any.whl size=261287 sha256=cb6231f0c60f436f0581274a9cda3750b4ffde2b30709a46db39931b969b8af3
  Stored in directory: /tmp/pip-ephem-wheel-cache-pxb0_0bg/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=6444db3f600e90052a2018424aa67e5876ee6946e5f2bd40eb60a2f58ed96412
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14

In [9]:
import pandas_profiling # Now we load the package
insurance2.profile_report() # And we call the package. This will take a moment. Prepare to be amazed!

ImportError: ignored

##**1.3 Setting up Training and Test Sets**
To set up our training and test sets, we first split the independent variables (which we are assuming are age, bmi, children, and charges) and the class attribute (region), which contains the labels that we want to assign to the "unknown" data.



In [10]:
x=insurance2.iloc[:,:4] # all parameters
y=insurance2['region'] # class labels 'southwest', 'southeast', 'northwest', 'northeast'

#print(x) # Uncomment this line to verify your parameters/ independent variables/ attributes/ features
#print(y) # Uncomment this line to verify your class labels

Now that we have separated the x attributes (independent variables) and the y attribute (class attribute, dependent variable), we build our training and test sets! 

NOTE that we are not allocating any sizes for the train_test_split below. This will invoke the default, which is a 75% training/ 25% test split.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 0)

# So, what training data do we have?
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

X_train shape: (1003, 4)
y_train shape: (1003,)


##Your Turn
How many rows and columns do you have in the test data set? Write the command below and run it!

In [12]:
X_train.head()

,age,bmi,children,charges
1075,32,29.59,1,4562.84210
131,61,22.04,0,13616.35860
15,19,24.60,1,1837.23700
1223,20,24.42,0,26125.67477
1137,26,22.23,0,3176.28770


##**1.4 Building the Simplest Model with k=1**

Remember that, in kNN classification, the output is class membership. An object is classified by a majority vote of its neighbors, with the object being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor.

We'll try that out first.[link text](https://)

In [13]:
# This is the model with the one nearest neighbor. Note the output shows that the metric is set to minkowski, 
# but the p is set to 2, which means that we are using Euclidian distance (1 = Manhattan distance)

from sklearn.neighbors import KNeighborsClassifier
model1 = KNeighborsClassifier(n_neighbors = 1)
model1.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

###**1.4.1 Testing the Model**

As you can see, we have built model1, which is the kNN model with just 1 nearest neighbor. Next, we test it.

We use y_pred to store the calculated y values (remember y hat?) that the model gives us. Then we can compare them with the actual y values that we know and see what percentage the model identified correctly.

In [14]:
y_pred = model1.predict(X_test)
print("Test set predictions: \n {}".format(y_pred))

Test set predictions: 
 ['northeast' 'southwest' 'southwest' 'southeast' 'southwest' 'northwest'
 'northeast' 'southwest' 'northwest' 'southeast' 'northeast' 'southwest'
 'southwest' 'southeast' 'southeast' 'southwest' 'northwest' 'southwest'
 'northwest' 'northwest' 'northwest' 'southwest' 'southwest' 'southwest'
 'southwest' 'southeast' 'northwest' 'northeast' 'southeast' 'northwest'
 'southwest' 'southeast' 'southeast' 'northwest' 'northeast' 'northwest'
 'northeast' 'southeast' 'northwest' 'southeast' 'northeast' 'southwest'
 'southwest' 'southeast' 'southeast' 'southeast' 'southeast' 'northwest'
 'northeast' 'southeast' 'northeast' 'northwest' 'southeast' 'southeast'
 'southeast' 'southwest' 'southwest' 'southwest' 'southeast' 'northeast'
 'southwest' 'northwest' 'northwest' 'southwest' 'southwest' 'southeast'
 'northwest' 'southwest' 'northwest' 'southeast' 'northeast' 'southwest'
 'southwest' 'southeast' 'southeast' 'northeast' 'northeast' 'northwest'
 'northeast' 'southeast' 'n

###**1.4.2 Evaluating the model**
Here, we use three critical methods to get an idea of how "good" our model really is.


####1.4.2.1 The Accuracy Score

In [15]:
# Accuracy score
print("Test set score: {:.2f}".format(model1.score(X_test, y_test)))

Test set score: 0.36


Oh boy! That number could be better. Let's see if the other methods indicate the same thing.

####1.4.2.2 Data Inspection Calculated vs. Actual values

In [16]:
# Let's compare the actual y and the predicted y

realvsmodel1 = pd.DataFrame(y_pred,y_test)
realvsmodel1 = pd.DataFrame({'predicted':y_pred,'original':y_test})
realvsmodel1.head()

,predicted,original
578,northeast,southwest
610,southwest,southeast
569,southwest,northwest
1034,southeast,northwest
198,southwest,northwest


Well, that doesn't look too exciting, does it? Let's go to the last and MOST IMPORTANT way of evaluating the quality of our model: **The Confusion Matrix.** 

####1.4.2.3 The Confusion Matrix

A confusion matrix compares the calculated or predicted values for all labels in the class attribute with the actual, true values that we know. In other words, we check which true values were predicted correctly and which were predicted incorrecty. For this purpose, we will use the plot_confusion_matrix group from the scikit-learn.metrics library.

A longer, more mathematical, explanation is [here](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/).

In [17]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(model1, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


In [18]:
plot_confusion_matrix(model1, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


Alrighty. What are we seeing here? 

The true values appear in rows (label on the left side, down), the predicted values appear in columns (label across the top). Here is how to explain **the first row**: 
* All the items should have been in the northeast region because the True Label is 'northeast'. That is clearly not the case; otherwise, we would see 96 in the green box in the upper left and zeros across the rest of the row.
* Of the actual true 'northeast' region, 35 were predicted correctly (green); 18 were incorrectly predicted as 'northwest', 17 were incorrectly predicted as 'southeast', and 26 were incorrectly predicted as 'southwest'. So, out of 96 actual 'northeast' rows, only 35 were predicted correctly. 18+17+26 (= 61) were predicted incorrectly.

###1.4.2.4 The Classification Report

The Classification Report gives us even more insights into how well (or, in our case, badly) our model performs. To read it correctly, we first have to define a few terms:
1. **precision** (also called positive predictive value) is the number of correctly identified positive results divided by the number of all positive results, including those not identified correctly ((true positives) / (true positives + false positives)). Said another way, “for all instances classified positive, what percent was correct?”
2. **recall** (also known as sensitivity) is the number of correctly identified positive results divided by the number of all samples that should have been identified as positive ((true positives) / (true positives + false negatives)). Said another way, “for all instances that were actually positive, what percent was classified correctly?
3. **f-1 score** is the harmonic mean of the precision and recall. The highest possible value of F1 is 1, indicating perfect precision and recall, and the lowest possible value is 0, if either the precision or the recall is zero. As a rule of thumb, the weighted average of F1 should be used to compare classifier models, not global accuracy.
3. **support** is the number of actual occurrences of the class in the specified dataset.

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, labels=['southwest', 'southeast', 'northwest', 'northeast']))

              precision    recall  f1-score   support

   southwest       0.18      0.23      0.21        64
   southeast       0.42      0.43      0.43        88
   northwest       0.43      0.38      0.40        87
   northeast       0.41      0.36      0.38        96

    accuracy                           0.36       335
   macro avg       0.36      0.35      0.35       335
weighted avg       0.37      0.36      0.37       335



Look at these values (note that "support" is the number of all true label quantities)--would you accept this quality from, say, a dentist? Or from your car, which (in the case of southwest), engages the brakes in 18% of all cases when you step on the pedal? 

Yeah, I thought so, too. Not good.

## **1.5 Building kNN with 5 nearest neighbors (and a 2/3 to 1/3 train/ test split)**
Given our less-than fabulous results above, let's see if instead of assigning the class label from only 1 nearest neighbor, we can increase the accuracy of our predictions by looking at the class labels for the 5 nearest neighbors!

###1.5.1 Setting up Training and Test Set
Note how here, we use the test_size parameter to split 2/3 of the data into the training set and 1/3 into the test set.

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.33)

print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (896, 4)
y_train shape: (896,)
X_test shape: (442, 4)
y_test shape: (442,)


###1.5.2 Building the model with k=5

In [21]:
model5 = neighbors.KNeighborsClassifier(n_neighbors=5)

model5.fit(X_train, y_train)

y_pred = model5.predict(X_test)

Alright, we've got the model built. Everyone ready for the Confusion Matrix and the Classification Report?

In [22]:
plot_confusion_matrix(model5, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


In [23]:
print(classification_report(y_test, y_pred, labels=['southwest', 'southeast', 'northwest', 'northeast']))

              precision    recall  f1-score   support

   southwest       0.24      0.10      0.14       110
   southeast       0.35      0.35      0.35       123
   northwest       0.34      0.39      0.36       114
   northeast       0.28      0.41      0.33        95

    accuracy                           0.31       442
   macro avg       0.30      0.31      0.30       442
weighted avg       0.30      0.31      0.30       442



Wait, seriously? Why are we getting a result that is only a little bit better, but we're spending much more processing effort? There's got to be a way to find the optimal number for k!

And there is.

##**1.6 Optimizing k with Cross-Validation**
We could spend entire days re-running the kNN and increasing k by 1 until we've found the best value for k. But that would be 1. boring, 2. too much work, 3. not efficient, given that we could instead just cycle through a list of values until we've found the best one. 

To achieve this in sa most efficient way, we will need to use another trick (aka a preprocessing method) that we haven't encountered yet: **Cross-validation.** Find out [in this detail description](https://machinelearningmastery.com/k-fold-cross-validation/) how cross-validation works. [This graphic](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png) will also help you understand. Or watch this 5-minute video:

In [24]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/fSytzGwwBVw" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

This what we'll do now. Let's go!

First, we build a list of potential k values. Then we create an empty list that will hold cross-validation scores.

In [25]:
# To determine how to pick k, we are first creating a list of potential k values
klist = list(range(1,50,2)) # Our list goes from 1 to 50 in increments of 2

# Then we create an empty list that will hold cross-validation scores
cv_scores = []

Now we can build our cross-validation. We will cycle through our k values from the k-value list and store the accuracy scores in the cv_scores list. To make things easier on us, we will convert the accuracy score into its opposite--the misclassification error. This misclassification error is really the average of all the misclassifications for one run of k.

In [26]:
# Perform 10-fold cross validation for each k value (we have a small dataset, so we can do this)
for k in klist:
    model10 = neighbors.KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model10, x, y, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# Changing to misclassification error
errors = 1- np.array(cv_scores)

NOW we can use the error number to determine the optimal k! To do so, we look at our errors and pick the row with the k value that produced the smallest error. 

To make things easier to understand, we plot the misclassification errors in comparison to k so we can see our results.

In [27]:
optimal_k = klist[np.argmin(errors)]
print("The optimal number of neighbors is {}".format(optimal_k))

# plot misclassification error vs k
plt.plot(klist, errors)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

The optimal number of neighbors is 1


##Your Turn
Re-run this model with k=10 and k=50; for each of these, build and run a confusion matrix and a classification report. What changes? What do your results say about the data?

Use the lines below for your code

In [28]:
# Perform 10-fold cross validation for each k value (we have a small dataset, so we can do this)
for k in klist:
    model10 = neighbors.KNeighborsClassifier(n_neighbors=10)
    scores = cross_val_score(model10, x, y, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# Changing to misclassification error
errors = 1- np.array(cv_scores)

In [29]:
# Perform 10-fold cross validation for each k value (we have a small dataset, so we can do this)
for k in klist:
    model10 = neighbors.KNeighborsClassifier(n_neighbors=50)
    scores = cross_val_score(model10, x, y, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# Changing to misclassification error
errors = 1- np.array(cv_scores)

# **2. Naive Bayes**
Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ theorem with the “naive” assumption of conditional independence between every pair of features given the value of the class variable. 

Here is a [great explanation](https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn) of the principle behind the Bayes Theorem. Read the webpage so you understand what we're up to--if you skip it, nothing below will make sense to you.

**WAIT!**
Read that first paragraph again. Then go all the way back to the Exploratory Data Analysis (EDA) section of this file. Look closely at the output of the pandas_profile analysis. Read the first paragraph another time. 


##Your Turn
Given the results from the EDA, should we conduct a Naive Bayes Analysis, at all? What condition does the insurance2 dataset violate? Type your answer below.


##**2.1 Setting up the Environment**
We will be working with Scikit-Learn again. More specifically, we will be working with the Gaussian Naive Bayes algorithm. Are there other Naive Bayes algorithms? [Absolutely](https://scikit-learn.org/stable/modules/naive_bayes.html). 

In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [32]:
#Let's verify that the dataset is still what it needs to be:
insurance2.head()

,age,bmi,children,charges,region
0,19,27.900,0,16884.92400,southwest
1,18,33.770,1,1725.55230,southeast
2,28,33.000,3,4449.46200,southeast
3,33,22.705,0,21984.47061,northwest
4,32,28.880,0,3866.85520,northwest


In [33]:
insurance2.dtypes

age           int64
bmi         float64
children      int64
charges     float64
region       object
dtype: object

##**2.2 Setting up the training and test sets**

In [34]:
ins_train, ins_test = train_test_split(insurance2, test_size = 0.2)
print(ins_train)
print(ins_test)

      age     bmi  children      charges     region
578    52  30.200         1   9724.53000  southwest
552    62  21.400         0  12957.11800  southwest
73     58  32.010         1  11946.62590  southeast
238    19  29.070         0  17352.68030  northwest
485    31  31.065         0   4347.02335  northeast
...   ...     ...       ...          ...        ...
231    59  27.830         3  14001.28670  southeast
896    43  20.045         2  19798.05455  northeast
1173   38  29.260         2   6457.84340  northwest
1092   23  32.700         3   3591.48000  southwest
166    20  37.000         5   4830.63000  southwest

[1070 rows x 5 columns]
      age     bmi  children      charges     region
818    47  26.125         1  23401.30575  northeast
310    50  26.600         0   8444.47400  southwest
617    49  25.600         2  23306.54700  southwest
1062   59  41.140         1  48970.24760  southeast
1026   19  26.030         1  16450.89470  northwest
...   ...     ...       ...          ..

In [35]:
ins_train_np = np.array([ins_train])
ins_test_np = np.array([ins_test])

##**2.3 Building the model with GaussianNB()**

In [36]:
ins_naivebayes = GaussianNB()
ins_naivebayes

GaussianNB()

In [37]:
ins_train.drop('region',axis=1)

,age,bmi,children,charges
578,52,30.200,1,9724.53000
552,62,21.400,0,12957.11800
73,58,32.010,1,11946.62590
238,19,29.070,0,17352.68030
485,31,31.065,0,4347.02335
...,...,...,...,...
231,59,27.830,3,14001.28670
896,43,20.045,2,19798.05455
1173,38,29.260,2,6457.84340
1092,23,32.700,3,3591.48000


In [38]:
ins_train['region']

578     southwest
552     southwest
73      southeast
238     northwest
485     northeast
          ...    
231     southeast
896     northeast
1173    northwest
1092    southwest
166     southwest
Name: region, Length: 1070, dtype: object

In [39]:
ins_naivebayes.fit(ins_train.drop('region',axis=1), ins_train['region'])

GaussianNB()

##**2.4 Testing the model and calculating the accuracy score**

In [40]:
ins_predictions = ins_naivebayes.predict(ins_test.drop('region',axis=1))
accuracy_score(ins_test['region'], ins_predictions)

0.34701492537313433

##**2.5 Comparing the predicted and the original values**

In [41]:
realvsmodel2 = pd.DataFrame(ins_predictions,ins_test)
realvsmodel2 = pd.DataFrame({'predicted':ins_predictions,'original':ins_test['region']})
realvsmodel2.head()

,predicted,original
818,northwest,northeast
310,northwest,southwest
617,northwest,southwest
1062,southeast,southeast
1026,northwest,northwest


##Your Turn
Plot a confusion matrix. For your code, refer to secion 1.4.2.3 above:

In [42]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(model1, X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator.
  warnings.warn(msg, category=FutureWarning)


## Your Turn
Build a classification report. For your code, refer to section 1.4.2.4 above:

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, labels=['southwest', 'southeast', 'northwest', 'northeast']))

              precision    recall  f1-score   support

   southwest       0.24      0.10      0.14       110
   southeast       0.35      0.35      0.35       123
   northwest       0.34      0.39      0.36       114
   northeast       0.28      0.41      0.33        95

    accuracy                           0.31       442
   macro avg       0.30      0.31      0.30       442
weighted avg       0.30      0.31      0.30       442



##Your Turn
Compare the results from the confusion matrix and the classification report for k Nearest Neighbor and Naive Bayes. Which model produces better results? Write your answer into the text field below.

##Your Turn
How valid are the results for the Naive Bayes classification really? Review the assumptions for work with Naive Bayes, especially regarding dependency among independent attribtues, and then look again at the results of the pandas_profiling output above. Write your answer into the text field below.



##Your Turn
So ... given the data that we have, can we reliably predict the region someone lives in based on their age, bmi, number of children, and the $ amount of their insurance claims? Explain in the field below.